Given the raw result files generate a final cleaned version of results:
  - Only the latest results; 
      - not any jobs which failed because of the benchmark framework and were redone, or
      - frameworks which were later excluded because issues were identified with the integration itself.
  - Transfer `RandomForest` results from 1 hour to 4 hour if 1 hour jobs ran to completion.
  - Impute `TunedRandomForest` results with random forest results of the same budget.
  

In [17]:
import pandas as pd

def filter_results(results):
    results = results.sort_values(by="utc", na_position="first")
    # There was a mistake in the old KDDCup09-Upselling task, so it was replaced with a new task.
    results = results[results["id"] != "openml.org/t/360947"]
    # Use only the latest results (earlier failures don't count, only justified reruns are done)
    results = results.drop_duplicates(["framework", "task", "fold"], keep="last")
    results = results[~results["framework"].isin(["autoxgboost", "GAMA", "MLPlanWEKA"])]
    return results


In [18]:
# Pick the results to
ttype = "regression"
ttype = "classification"

In [19]:
one = pd.read_csv(r"http://openml-test.win.tue.nl/amlb/{}_1h8c.csv".format(ttype))
one = filter_results(one)
four = pd.read_csv(r"http://openml-test.win.tue.nl/amlb/{}_4h8c.csv".format(ttype))
four = filter_results(four)


First a sanity check that 1H RF has results for every job (even if it is not a full forest)

In [20]:
rf = one[one.framework == "RandomForest"]
assert len(rf) == (330 if ttype == "regression" else 710)
assert rf["info"].isna().all()

Impute one hour Tuned Random Forest with Random Forest:

In [21]:
def impute_trf_with_rf(results):
    rf = results[results.framework == "RandomForest"]
    trf = results[results.framework == "TunedRandomForest"]
    missing_results = trf[~trf["info"].isna()][["task", "fold"]].itertuples(index=False, name=None)

    imputation_values = rf.set_index(["task", "fold"]).loc[missing_results].reset_index().copy()
    imputation_values["framework"] = "TunedRandomForest"

    trf_success = trf[trf["info"].isna()]
    trf_imputed = pd.concat([trf_success, imputation_values])

    no_trf = results[results.framework != "TunedRandomForest"]
    imputed = pd.concat([no_trf, trf_imputed])
    print(f"Original dataset with {len(results)} entries of which {sum(~results['info'].isna())} had missing results.")
    print(f"The new  dataset with {len(imputed)} entries of which {sum(~imputed['info'].isna())} are missing results because {len(imputation_values)} results were imputed.")
    return imputed

one_imputed = impute_trf_with_rf(one)    

Original dataset with 8519 entries of which 272 had missing results.
The new  dataset with 8519 entries of which 252 are missing results because 20 results were imputed.


Impute four hour Random Forest with complete one hour Random Forest:

In [22]:
rf_one = one[one.framework == "RandomForest"]
keep = rf_one[(rf_one["models_count"] == 2000.0) & (~rf_one["result"].isna())].copy()
keep["constraint"] = "4h8c_gp3"
four_added = pd.concat([four, keep])
print(f"Original dataset had {len(four):6d} entries of which {len(four[four.framework == 'RandomForest']):6d} were 4H RF.")
print(f"The new  dataset has {len(four_added):6d} entries of which {len(four_added[four_added.framework == 'RandomForest']):6d}  are 4H RF.")

Original dataset had   8570 entries of which     50 were 4H RF.
The new  dataset has   9230 entries of which    710  are 4H RF.


Above file is useful to avoid running 4H RF experiments which would grow the same (sized) forests as the 1H budget ones. This result file was also used to `--resume` from to automatically find the remaining 4H RF experiments.

After completing all RF results, we can use them to impute the `TunedRandomForest` where it otherwise failed.

In [23]:
four_imputed = impute_trf_with_rf(four_added)  

Original dataset with 9230 entries of which 451 had missing results.
The new  dataset with 9230 entries of which 427 are missing results because 24 results were imputed.


We only need to perform the `constantpredictor` baseline once, since the result is deterministic regardless of time budget. We performed the set of experiments with a four hour time budget and transfer the to one hour:

In [24]:
constants = four_imputed[four_imputed.framework == "constantpredictor"].copy()
constants["constraint"] = "1h8c_gp3"
one_imputed = pd.concat([one_imputed, constants])

In [25]:
# The constantpredictor experiments were ran without the gp3 SSD, but we can rename the result.
four_imputed.loc[four_imputed.framework == "constantpredictor", "constraint"] = "4h8c_gp3"

In [26]:
final = pd.concat([one_imputed, four_imputed])

A few sanity checks:

In [27]:
if ttype == "classification":
    jobs = 71 * 10 * 13 * 2 - 1 # tasks * folds * frameworks * time budgets - known failures
else:
    jobs = 33 * 10 * 12 * 2 # autosklearn2 does not support regression

assert len(final) == jobs
assert len(final) == len(final.drop_duplicates(["framework", "task", "fold", "constraint"]))

In [28]:
final.to_csv(f"{ttype}_all_cleaned.csv", index=False)